In [21]:
from lxml import etree
import abc
import re
import requests

import gensim.models as models
import gensim.models.word2vec as word2vec
import secure

import search
reload(search)
import rewriter
reload(rewriter)


In [1]:
corpus = word2vec.Text8Corpus(secure.DATASET_PATH_BASE + 'enwik8')
# self.model = word2vec.Word2Vec(corpus, workers=8)
# self.model.save(self.MODEL_PATH)

In [9]:
from gensim.models.phrases import *

ph = Phrases(corpus)

In [12]:
phr = Phraser(ph)

In [13]:
sent = [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']

In [14]:
print phr[sent]

[u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']


In [17]:
phr[[u'chemical',u'reactions',u'are',u'cool']]

[u'chemical_reactions', u'are', u'cool']

In [21]:
import warnings

In [22]:
import re
warnings.filterwarnings("ignore", message=re.escape("For a faster implementation, use the gensim.models.phrases.Phraser class"))

In [23]:
mod = word2vec.Word2Vec(ph[corpus],size=10)

In [26]:
mod.most_similar(positive=['person'], topn=10)

[(u'looking_forward', 0.9501240849494934),
 (u'compounds]', 0.9304075241088867),
 (u"'''[[Internet", 0.9185891151428223),
 (u'Marks]]', 0.9161316156387329),
 (u'(1995).', 0.9128404259681702),
 (u'anti-Communist', 0.9109591245651245),
 (u'Font', 0.9097691774368286),
 (u'[[National_Library', 0.9085487723350525),
 (u"'''F'''", 0.9075831174850464),
 (u'(Istat_2005)', 0.9073328971862793)]

In [27]:
from gensim.models.wrappers import FastText

In [195]:
# WARNING: takes like 10 min to train
# made from https://github.com/facebookresearch/fastText
path = '/Users/neel/Dropbox/Harvard/Classes/research/query-rewriter/fastText/fasttext'
fm = FastText.train(path, corpus_file=secure.DATASET_PATH_BASE + 'wikiclean8')

KeyboardInterrupt: 

In [61]:
fm.similar_by_word('new_york', topn=20)

[(u'Zork', 0.816493034362793),
 (u'Ankh-Morpork', 0.7960439920425415),
 (u'Artwork', 0.7774671316146851),
 (u'Network', 0.7542241811752319),
 (u'pork', 0.7466747760772705),
 (u'Clockwork', 0.7459302544593811),
 (u'homework', 0.7433512806892395),
 (u'Cork', 0.7420310974121094),
 (u'newsreel', 0.7417821884155273),
 (u'newline', 0.7417464256286621),
 (u'Pork', 0.7369546294212341),
 (u'(Network', 0.729056715965271),
 (u'company_slogan', 0.7259443998336792),
 (u'news]', 0.7191295623779297),
 (u'network', 0.7173770666122437),
 (u'new', 0.7120053172111511),
 (u'Yorkshire', 0.7098603248596191),
 (u'stonework', 0.7096399068832397),
 (u'new,', 0.7078135013580322),
 (u'artwork', 0.7072222232818604)]

In [4]:
from gensim.models.wrappers import FastText
import secure

In [64]:
fm.most_similar(positive=['new_york'], topn=20)

[(u'Zork', 0.816493034362793),
 (u'Ankh-Morpork', 0.7960439920425415),
 (u'Artwork', 0.7774671316146851),
 (u'Network', 0.7542241811752319),
 (u'pork', 0.7466747760772705),
 (u'Clockwork', 0.7459302544593811),
 (u'homework', 0.7433512806892395),
 (u'Cork', 0.7420310974121094),
 (u'newsreel', 0.7417821884155273),
 (u'newline', 0.7417464256286621),
 (u'Pork', 0.7369546294212341),
 (u'(Network', 0.729056715965271),
 (u'company_slogan', 0.7259443998336792),
 (u'news]', 0.7191295623779297),
 (u'network', 0.7173770666122437),
 (u'new', 0.7120053172111511),
 (u'Yorkshire', 0.7098603248596191),
 (u'stonework', 0.7096399068832397),
 (u'new,', 0.7078135013580322),
 (u'artwork', 0.7072222232818604)]

In [68]:
fm.save(secure.MODEL_PATH_BASE+'fasttext/fasttext')

In [5]:
fm2 = FastText.load(secure.MODEL_PATH_BASE+'fasttext/fasttext')

In [6]:
fm2.similar_by_word('chemistry', topn=25)

[(u'biochemistry', 0.9830586910247803),
 (u'chemistry|', 0.9796395897865295),
 (u'(chemistry)', 0.9524341821670532),
 (u'chemistry,', 0.9513406753540039),
 (u'chemistry==', 0.9481360912322998),
 (u'biochemistry,', 0.9481123685836792),
 (u"'Radiochemistry", 0.9445796608924866),
 (u'Biochemistry', 0.9373379945755005),
 (u'Electrochemistry', 0.9102224111557007),
 (u"chemistry'''", 0.9014816284179688),
 (u'chemist', 0.8953343629837036),
 (u'chemistry.', 0.8936431407928467),
 (u'[[biochemistry|biochemical]]', 0.8890479803085327),
 (u'electrochemistry', 0.8872667551040649),
 (u'biochemistry.', 0.8862274289131165),
 (u'chemist,', 0.8851226568222046),
 (u'biochemist', 0.8840389251708984),
 (u'biochemist,', 0.8771650195121765),
 (u'[[chemistry|chemical]]', 0.8727388381958008),
 (u'Chemistry', 0.8726977109909058),
 (u'ischemic', 0.8639161586761475),
 (u'alchemist', 0.8624183535575867),
 (u'physiology', 0.8568892478942871),
 (u'Chemistry,', 0.8566870093345642),
 (u'[[electrochemistry|electrochemi

In [77]:
fm2.init_ngrams()

IndexError: index 1962073 is out of bounds for size 1474803

In [169]:
corpus = word2vec.LineSentence(secure.DATASET_PATH_BASE + 'wikiclean8')

w2v = rewriter.Word2VecRewriter(corpus=None, create=False)

In [170]:
w2v.rewrite("ohio")

[u'michigan',
 u'iowa',
 u'wisconsin',
 u'illinois',
 u'pennsylvania',
 u'birmingham',
 u'oregon',
 u'arizona',
 u'nebraska',
 u'kentucky',
 u'ohio']

In [93]:
from gensim.models.phrases import *

phrases = Phrases(corpus)

In [51]:
import re
warnings.filterwarnings("ignore", message=re.escape("For a faster implementation, use the gensim.models.phrases.Phraser class"))

In [52]:
model_bigram = word2vec.Word2Vec(phrases[corpus])

In [53]:
# warning: you get a lot of gibberish! :(
model_bigram.similar_by_word("new_york")

[(u'tyrannosaurus', 0.48952293395996094),
 (u'geared', 0.46873939037323),
 (u'rosencrantz', 0.40396764874458313),
 (u'bittorrent', 0.38522869348526),
 (u'concludes', 0.38276174664497375),
 (u'gur', 0.38172149658203125),
 (u'schaff_herzog', 0.38074791431427),
 (u'scotus', 0.3685116171836853),
 (u'fair_dealing', 0.36827796697616577),
 (u'be_remembered', 0.36687690019607544)]

In [54]:
model = word2vec.Word2Vec.load(MODEL_PATH)

In [62]:
model.similar_by_word("learning")

[(u'teaching', 0.7068275213241577),
 (u'studying', 0.6510186791419983),
 (u'education', 0.6509798765182495),
 (u'communication', 0.6505078077316284),
 (u'skills', 0.6451709270477295),
 (u'expertise', 0.644391655921936),
 (u'knowledge', 0.6385884881019592),
 (u'curriculum', 0.6323326230049133),
 (u'creativity', 0.6304869055747986),
 (u'thinking', 0.6236695051193237)]

In [94]:
phraser = Phraser(phrases)

In [104]:
phraser["this is a chemical reaction".split(" ")]

[u'this', u'is', u'a', u'chemical_reaction']

In [165]:
import collections

z = 0

class PhraserCorpus(collections.Iterator):
    """
    An iterable that wraps over a Corpus and the Phraser. When word2vec
    requests a sentence from the corpus, this passes the sentence through the
    Phraser first.
    
    This makes it very easy to implement bigrams with just a vanilla Corpus
    and a simple Phraser.
    """
    def __init__(self, phraser, corpus_iter):
        self.corpus_iter = corpus_iter
        self.phraser = phraser
        
    def __iter__(self): return self
    
    def next(self):
        # wrap the corpus_iter.next() call with a call to the phraser 
        # if it throws a stop iteration
        try:
            sentence = self.corpus_iter.next()
            global z
            z += 1
            # apply the phraser here 
            return phraser[sentence]
        except StopIteration:
            # we should stop when it stops
            raise StopIteration

In [166]:
ci = iter(corpus)

In [167]:
pc = PhraserCorpus(phraser, ci)

In [168]:
mod3 = word2vec.Word2Vec(pc)

In [171]:
mode3

481196

In [172]:
mod3.similar_by_word("french_revolution")

[(u'diluted', 0.4184649884700775),
 (u'romantic_comedy', 0.37694448232650757),
 (u'shell', 0.3746798038482666),
 (u'bakunin', 0.37169894576072693),
 (u'dragonetti', 0.3654710352420807),
 (u'feasting', 0.3645687699317932),
 (u'candlemakers', 0.36218002438545227),
 (u'parish_churches', 0.3621782660484314),
 (u'innate_immunity', 0.35786914825439453),
 (u'suitable_for', 0.3556133210659027)]

In [173]:
import collections

class ControlCorpus(collections.Iterator):
    """
    An iterable that wraps over a Corpus and the Phraser. When word2vec
    requests a sentence from the corpus, this passes the sentence through the
    Phraser first.
    
    This makes it very easy to implement bigrams with just a vanilla Corpus
    and a simple Phraser.
    """
    def __init__(self, phraser, corpus_iter):
        self.corpus_iter = corpus_iter
        self.phraser = phraser
        
    def __iter__(self):
        try:
            while True:
                yield self.corpus_iter.next()
    
#     def next(self):
#         # wrap the corpus_iter.next() call with a call to the phraser 
#         # if it throws a stop iteration
#         try:
#             sentence = self.corpus_iter.next()
#             global z
#             z += 1
#             # apply the phraser here 
#             return sentence
#         except StopIteration:
#             # we should stop when it stops
#             raise StopIteration

In [174]:
ci2 = iter(corpus)

In [178]:
pd = ControlCorpus(phraser, ci2)

In [179]:
mod4 = word2vec.Word2Vec(pd)

In [182]:
mod4.similar_by_word("chemistry")

[(u'bipartisan', 0.40629392862319946),
 (u'mes', 0.3698464632034302),
 (u'uprisings', 0.3670933246612549),
 (u'mats', 0.3569819927215576),
 (u'woman', 0.3505661189556122),
 (u'impressionists', 0.34208574891090393),
 (u'relatives', 0.32833635807037354),
 (u'specified', 0.32693660259246826),
 (u'expiry', 0.32566046714782715),
 (u'arts', 0.32548588514328003)]

In [187]:
w2v.model.similar_by_word("chemistry")

[(u'biochemistry', 0.8456375002861023),
 (u'biology', 0.8456199169158936),
 (u'physics', 0.8300018310546875),
 (u'mathematics', 0.7767337560653687),
 (u'molecular', 0.774981677532196),
 (u'optics', 0.7711237072944641),
 (u'synthesis', 0.7402553558349609),
 (u'genetics', 0.7353801727294922),
 (u'botany', 0.7348920106887817),
 (u'zoology', 0.7271405458450317)]

In [189]:
phraser

In [190]:
corpus

In [191]:
ii = phraser[corpus]

In [192]:
ii

In [193]:
mod5 = word2vec.Word2Vec(ii)

In [194]:
mod5.similar_by_word("chemistry")

[(u'bipartisan', 0.40629392862319946),
 (u'strict_observance', 0.3852868974208832),
 (u'became_estranged', 0.37987571954727173),
 (u'juliet_barker', 0.3734053373336792),
 (u'team_finished', 0.37240421772003174),
 (u'motor_neuron', 0.3719816207885742),
 (u'thunder_bay', 0.3712693750858307),
 (u'mes', 0.3698464632034302),
 (u'contentious_issue', 0.3677879571914673),
 (u'uprisings', 0.3670933246612549)]

## Using a phraser doesn't work... sometimes

Word2Vec has a feature called a Phraser, which generates a corpus of bigrams out of text. But oddly, sometimes it just doesn't work. For instance, here's a toy example. First, we use the raw corpus and a normal word2vec model, and everything works fine...

In [233]:
corpus = word2vec.LineSentence(secure.DATASET_PATH_BASE + 'test8')

In [234]:
raw_model = word2vec.Word2Vec(corpus)

In [235]:
raw_model.similar_by_word("person")

[(u'from', 0.9997075796127319),
 (u'of', 0.9997063875198364),
 (u'this', 0.9997018575668335),
 (u'autism', 0.999697208404541),
 (u'anarchism', 0.9996892809867859),
 (u'in', 0.9996891617774963),
 (u'to', 0.9996873140335083),
 (u'and', 0.9996873140335083),
 (u'at', 0.9996867775917053),
 (u'an', 0.9996865391731262)]

Now, if we try using the Phraser class, we get total garbage (observe the drop from 99.97% relevance to 32%):

In [236]:
phrases = Phrases(corpus)

In [237]:
phraser = Phraser(phrases)

In [238]:
bigrammed_model = word2vec.Word2Vec(phraser[corpus])

In [239]:
bigrammed_model.similar_by_word("person")

[(u'example', 0.3241819441318512),
 (u'must', 0.2850297689437866),
 (u'now', 0.259639710187912),
 (u'point', 0.23865681886672974),
 (u'schools', 0.21188122034072876),
 (u'there_is', 0.2076726257801056),
 (u'true', 0.20666877925395966),
 (u'individual', 0.20091284811496735),
 (u'anarcha', 0.20034271478652954),
 (u'org', 0.1967254877090454)]

Even stranger still - if we convert the  phraser into a list and then feed it into the model, the model walks OK and almost the same as before. But why???

In [ ]:
phrase_list = list(phraser[corpus])

In [241]:
listed_model = word2vec.Word2Vec(phrase_list)

In [242]:
listed_model.similar_by_word("person")

[(u'of', 0.9997658729553223),
 (u'from', 0.9997580647468567),
 (u'in', 0.9997528195381165),
 (u'autism', 0.999750018119812),
 (u'to', 0.9997495412826538),
 (u'and', 0.9997493028640747),
 (u'have', 0.9997491836547852),
 (u'this', 0.9997477531433105),
 (u'anarchism', 0.9997474551200867),
 (u'a', 0.9997439980506897)]